In [1]:
# === Cell 1: Install required packages (run once) ===
%pip install passlib pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
# === Cell 2: Imports and configuration ===
import os
import logging
from dataclasses import dataclass
from datetime import datetime
from time import perf_counter
from typing import Tuple, Optional, List, Dict

from pymongo import MongoClient, ASCENDING
from pymongo.errors import DuplicateKeyError
from passlib.hash import scrypt, pbkdf2_sha256
import secrets  # for secure randomization (tokens, etc.)


In [3]:
# === Cell 2b: Config class ===
@dataclass
class Config:
    # MongoDB connection details
    MONGO_URI: str = os.environ.get("MONGO_URI", "mongodb://localhost:27017/")
    DB_NAME: str = os.environ.get("DB_NAME", "secure_password_db")
    USER_COLLECTION: str = os.environ.get("USER_COLLECTION", "users")

    # Hashing parameters (tunable for performance/security)
    SCRYPT_N: int = int(os.environ.get("SCRYPT_N", 2**14))   # CPU/memory cost
    SCRYPT_R: int = int(os.environ.get("SCRYPT_R", 8))
    SCRYPT_P: int = int(os.environ.get("SCRYPT_P", 1))
    PBKDF2_ROUNDS: int = int(os.environ.get("PBKDF2_ROUNDS", 200_000))


In [4]:
# === Cell 3: Logging configuration ===
logger = logging.getLogger("secure_password_system")
logger.setLevel(logging.INFO)

# Avoid duplicate handlers if you re-run the cell
if not logger.handlers:
    formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")

    # Console handler
    ch = logging.StreamHandler()
    ch.setFormatter(formatter)
    logger.addHandler(ch)

    # File handler (logs to auth.log in current folder)
    fh = logging.FileHandler("auth.log", encoding="utf-8")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

logger.info("Logger initialised.")


2025-11-04 03:27:15,050 [INFO] Logger initialised.


In [5]:
# === Cell 4: Database manager with MongoDB + in-memory fallback ===
from pymongo.errors import DuplicateKeyError, ServerSelectionTimeoutError

class DatabaseManager:
    def __init__(self, cfg: Config, use_memory_fallback: bool = True):
        self.cfg = cfg
        self.in_memory = False
        self._mem_users = {}

        try:
            # small timeout so it fails fast if Mongo is down
            self.client = MongoClient(cfg.MONGO_URI, serverSelectionTimeoutMS=3000)
            # "ping" the server to ensure it is reachable
            self.client.admin.command("ping")

            self.db = self.client[cfg.DB_NAME]
            self.coll = self.db[cfg.USER_COLLECTION]
            self.coll.create_index([("email", ASCENDING)], unique=True)
            logger.info("Connected to MongoDB at %s", cfg.MONGO_URI)
        except ServerSelectionTimeoutError as e:
            if not use_memory_fallback:
                raise
            logger.warning("MongoDB not available, using in-memory database. Reason: %s", e)
            self.client = None
            self.db = None
            self.coll = None
            self.in_memory = True

    # ----- helper methods that work for both backends -----
    def insert_user(self, user_doc: dict) -> Tuple[bool, Optional[str]]:
        if self.in_memory:
            email = user_doc["email"]
            if email in self._mem_users:
                return False, "Email already exists"
            self._mem_users[email] = user_doc
            return True, None
        else:
            try:
                self.coll.insert_one(user_doc)
                return True, None
            except DuplicateKeyError:
                return False, "Email already exists"

    def find_user(self, email: str) -> Optional[dict]:
        if self.in_memory:
            return self._mem_users.get(email)
        else:
            return self.coll.find_one({"email": email})

    def list_users(self) -> List[Dict]:
        if self.in_memory:
            return [
                {
                    "email": u["email"],
                    "algorithm": u["algorithm"],
                    "created_at": u["created_at"],
                }
                for u in self._mem_users.values()
            ]
        else:
            return list(
                self.coll.find(
                    {},
                    {"_id": 0, "email": 1, "algorithm": 1, "created_at": 1},
                )
            )

    def clear_all_users(self):
        if self.in_memory:
            self._mem_users.clear()
            logger.info("Cleared all users from in-memory database.")
        else:
            self.coll.delete_many({})
            logger.info("Cleared all users from MongoDB collection '%s'.", self.cfg.USER_COLLECTION)


In [6]:
# === Cell 5: Authentication manager (fixed scrypt usage) ===
class AuthManager:
    def __init__(self, cfg: Config):
        self.cfg = cfg

    def hash_password(self, password: str, algorithm: str = "scrypt") -> str:
        algo = algorithm.lower()
        if algo == "scrypt":
            # Use passlib's default scrypt parameters (no n/r/p here)
            hashed = scrypt.hash(password)
        elif algo in ("pbkdf2", "pbkdf2_sha256"):
            hashed = pbkdf2_sha256.using(
                rounds=self.cfg.PBKDF2_ROUNDS
            ).hash(password)
        else:
            raise ValueError(f"Unsupported algorithm: {algorithm}")
        return hashed

    def verify_password(self, password: str, hash_str: str, algorithm: str) -> bool:
        algo = algorithm.lower()
        if algo == "scrypt":
            return scrypt.verify(password, hash_str)
        elif algo in ("pbkdf2", "pbkdf2_sha256"):
            return pbkdf2_sha256.verify(password, hash_str)
        else:
            raise ValueError(f"Unsupported algorithm: {algorithm}")

    def build_user_document(
        self, email: str, password: str, algorithm: str = "scrypt"
    ) -> dict:
        hashed = self.hash_password(password, algorithm)
        # optional secure random token (e.g., for account activation)
        activation_token = secrets.token_urlsafe(16)
        return {
            "email": email,
            "password": hashed,
            "algorithm": algorithm.lower(),
            "created_at": datetime.utcnow(),
            "activation_token": activation_token,
        }


In [7]:
# === Cell 6: Create config, DB and auth instances ===
cfg = Config()
db = DatabaseManager(cfg, use_memory_fallback=True)  # True = auto fallback to memory
auth = AuthManager(cfg)

backend = "MongoDB" if not db.in_memory else "in-memory store"
logger.info("System initialised using %s backend.", backend)
print(f"Backend in use: {backend}")


2025-11-04 03:27:19,664 [INFO] Connected to MongoDB at mongodb://localhost:27017/
2025-11-04 03:27:19,666 [INFO] System initialised using MongoDB backend.


Backend in use: MongoDB


In [8]:
# === Cell 7: Application-level functions ===
from getpass import getpass

def register_user(email: str, password: str, algorithm: str = "scrypt") -> Tuple[bool, str]:
    user_doc = auth.build_user_document(email, password, algorithm)
    ok, err = db.insert_user(user_doc)
    if ok:
        msg = f"User '{email}' registered with {algorithm}."
        logger.info(msg)
        return True, msg
    else:
        msg = f"Failed to register '{email}': {err}"
        logger.warning(msg)
        return False, msg


def login_user(email: str, password: str) -> Tuple[bool, str]:
    user = db.find_user(email)
    if not user:
        msg = "User not found."
        logger.warning("Login failed for %s: %s", email, msg)
        return False, msg

    algorithm = user["algorithm"]
    if auth.verify_password(password, user["password"], algorithm):
        msg = "Login successful."
        logger.info("Login successful for %s using %s.", email, algorithm)
        return True, msg
    else:
        msg = "Invalid password."
        logger.warning("Login failed for %s: wrong password.", email)
        return False, msg


def list_users() -> List[Dict]:
    users = db.list_users()
    logger.info("Listed %d users.", len(users))
    return users


In [9]:
# === Cell 8: Interactive validation, testing & performance comparison ===
from time import perf_counter
import getpass

def print_backend_info():
    backend = "MongoDB" if not getattr(db, "in_memory", False) else "in-memory store"
    print("Backend in use:", backend)
    if not getattr(db, "in_memory", False):
        print("MongoDB URI:", cfg.MONGO_URI)
        print("Database name:", cfg.DB_NAME)
        print("Collection name:", cfg.USER_COLLECTION)
    else:
        print("Records are kept only in the notebook memory (_not_ persisted).")

def prompt_yes_no(msg: str, default: bool = True) -> bool:
    d = "Y/n" if default else "y/N"
    ans = input(f"{msg} [{d}]: ").strip().lower()
    if ans == "":
        return default
    return ans[0] == "y"

def compare_algorithms_for_db_users(loops: int = 100):
    """
    Compare scrypt vs PBKDF2 for the users currently stored in the DB.
    For each user, you enter the correct password once (input hidden),
    and we measure avg hash & verify time using that real password.
    """
    # fetch all user docs with email, algorithm, password (hash)
    if getattr(db, "in_memory", False):
        docs = list(db._mem_users.values())
    else:
        docs = list(db.coll.find({}, {"_id": 0, "email": 1, "algorithm": 1, "password": 1}))

    if not docs:
        print("No users found in the database. Register some users first.")
        return

    print(f"\nFound {len(docs)} users in the database.")
    print("For each user, you will be asked to enter the CORRECT password (input is hidden).")
    print("Timings will be measured using that real password and the stored hash.\n")

    rows = []

    for doc in docs:
        email = doc["email"]
        algo = doc["algorithm"]
        hash_str = doc["password"]

        print(f"\n--- Testing user: {email} (algorithm: {algo}) ---")
        pwd = getpass.getpass("Enter the correct password for this user: ")

        if not pwd:
            print("No password entered, skipping this user.")
            continue

        # measure hashing time (creating new hashes with same algorithm)
        hash_times = []
        for _ in range(loops):
            t0 = perf_counter()
            auth.hash_password(pwd, algo)
            t1 = perf_counter()
            hash_times.append(t1 - t0)
        hash_avg = sum(hash_times) / len(hash_times)

        # measure verification time (verifying stored hash)
        verify_times = []
        for _ in range(loops):
            t0 = perf_counter()
            auth.verify_password(pwd, hash_str, algo)
            t1 = perf_counter()
            verify_times.append(t1 - t0)
        verify_avg = sum(verify_times) / len(verify_times)

        rows.append((email, algo, hash_avg, verify_avg))

    # print summary table
    print("\n=== Average timing over", loops, "runs per user (seconds) ===")
    print(f"{'Email':25} {'Algo':8} {'Hash_time':>12} {'Verify_time':>12}")
    for email, algo, ht, vt in rows:
        print(f"{email[:25]:25} {algo:8} {ht:12.4f} {vt:12.4f}")

def interactive_tests():
    print("== Interactive Validation, Testing & Comparison ==")
    print_backend_info()
    print("\nYou can perform:")
    print("  [R]egister user")
    print("  [L]ogin user")
    print("  [S]how users (safe view)")
    print("  [C]lear all users")
    print("  [T]iming comparison for stored users (scrypt vs pbkdf2)")
    print("  [Q]uit")

    if prompt_yes_no("Start from a clean collection (delete all users)?", default=False):
        db.clear_all_users()

    while True:
        cmd = input("\nEnter command (R/L/S/C/T/Q): ").strip().lower()
        if cmd == "r":
            # Register path
            email = input("Email: ").strip()
            if not email:
                print("Email required. Try again.")
                continue
            password = getpass.getpass("Password (input hidden): ")
            if not password:
                print("Password required. Try again.")
                continue
            algo = input("Algorithm (scrypt / pbkdf2) [scrypt]: ").strip().lower() or "scrypt"
            if algo not in ("scrypt", "pbkdf2"):
                print("Invalid algorithm. Using scrypt.")
                algo = "scrypt"
            ok, msg = register_user(email, password, algo)
            print(msg)
            if ok:
                print("-> Example: You can view stored users using command 'S'.")
        elif cmd == "l":
            # Login path
            email = input("Email: ").strip()
            if not email:
                print("Email required. Try again.")
                continue
            password = getpass.getpass("Password (input hidden): ")
            ok, msg = login_user(email, password)
            print(msg)
        elif cmd == "s":
            # Show users (safe view)
            users = list_users()
            if not users:
                print("No users found.")
            else:
                print(f"Stored users ({len(users)}):")
                for u in users:
                    print(u)
            # Optional: inspect raw DB including hashes
            if prompt_yes_no("Print raw DB docs including hashed passwords? (not recommended)", default=False):
                if getattr(db, "in_memory", False):
                    for v in db._mem_users.values():
                        print(v)
                else:
                    for doc in db.coll.find({}, {"_id": 0}):
                        print(doc)
        elif cmd == "c":
            if prompt_yes_no("Are you sure you want to delete ALL users?", default=False):
                db.clear_all_users()
                print("All users deleted.")
            else:
                print("Abort clear.")
        elif cmd == "t":
            # Timing comparison for stored users
            compare_algorithms_for_db_users()
        elif cmd == "q":
            print("Exiting interactive testing.")
            break
        else:
            print("Unknown command. Use R/L/S/C/T/Q.")

# Run interactive test helper
interactive_tests()


== Interactive Validation, Testing & Comparison ==
Backend in use: MongoDB
MongoDB URI: mongodb://localhost:27017/
Database name: secure_password_db
Collection name: users

You can perform:
  [R]egister user
  [L]ogin user
  [S]how users (safe view)
  [C]lear all users
  [T]iming comparison for stored users (scrypt vs pbkdf2)
  [Q]uit


Start from a clean collection (delete all users)? [y/N]:  n

Enter command (R/L/S/C/T/Q):  q


Exiting interactive testing.


## GUI

In [10]:
%pip install customtkinter


Note: you may need to restart the kernel to use updated packages.


In [13]:
# === Enhanced GUI for Secure Password Storage ===
import tkinter as tk
from tkinter import ttk, messagebox
from time import sleep
import threading

def open_gui():
    root = tk.Tk()
    root.title("🔒 Secure Password Storage System")
    root.geometry("480x430")
    root.resizable(False, False)
    root.configure(bg="#1e1e1e")

    # Apply a modern ttk theme
    style = ttk.Style()
    style.theme_use("clam")
    style.configure("TButton",
                    background="#0078D7", foreground="white",
                    font=("Segoe UI", 10, "bold"),
                    padding=6)
    style.map("TButton",
              background=[("active", "#005A9E")])
    style.configure("TLabel", background="#1e1e1e", foreground="white", font=("Segoe UI", 10))
    style.configure("TEntry", fieldbackground="#2d2d2d", foreground="white")

    # connection status (MongoDB)
    backend_label = ttk.Label(root, text="", font=("Segoe UI", 9, "italic"))
    backend_label.pack(pady=(6, 4))

    backend = "MongoDB ✅" if not getattr(db, "in_memory", False) else "In-memory (Temporary)"
    backend_label.config(text=f"Connected backend: {backend}")

    # Frame for content
    frame = tk.Frame(root, bg="#252526")
    frame.place(relx=0.5, rely=0.5, anchor="center")

    # Title label
    title_lbl = tk.Label(frame, text="🔐 Secure Password Manager", font=("Segoe UI", 14, "bold"), fg="#00BFFF", bg="#252526")
    title_lbl.grid(row=0, column=0, columnspan=3, pady=(15, 10))

    # Mode toggle (Register / Login)
    mode = tk.StringVar(value="register")
    def on_mode_change():
        if mode.get() == "register":
            algo_label.grid(row=5, column=0, padx=10, pady=5, sticky="e")
            algo_menu.grid(row=5, column=1, padx=10, pady=5, sticky="w")
            submit_btn.config(text="Register User")
        else:
            algo_label.grid_remove()
            algo_menu.grid_remove()
            submit_btn.config(text="Login User")

    tk.Radiobutton(frame, text="Register", variable=mode, value="register",
                   bg="#252526", fg="white", selectcolor="#0078D7", command=on_mode_change).grid(row=1, column=0, padx=10, pady=5, sticky="e")
    tk.Radiobutton(frame, text="Login", variable=mode, value="login",
                   bg="#252526", fg="white", selectcolor="#0078D7", command=on_mode_change).grid(row=1, column=1, padx=10, pady=5, sticky="w")

    # Email field
    ttk.Label(frame, text="Email:").grid(row=2, column=0, padx=10, pady=8, sticky="e")
    email_entry = ttk.Entry(frame, width=30)
    email_entry.grid(row=2, column=1, padx=10, pady=8, sticky="w")

    # Password field
    ttk.Label(frame, text="Password:").grid(row=3, column=0, padx=10, pady=8, sticky="e")
    pwd_entry = ttk.Entry(frame, width=30, show="*")
    pwd_entry.grid(row=3, column=1, padx=10, pady=8, sticky="w")

    # Password visibility toggle
    show_pwd = tk.BooleanVar(value=False)
    def toggle_password():
        pwd_entry.config(show="" if show_pwd.get() else "*")
        toggle_btn.config(text="🙈" if show_pwd.get() else "👁️")

    toggle_btn = ttk.Button(frame, text="👁️", width=3, command=toggle_password)
    toggle_btn.grid(row=3, column=2, padx=(0, 10), pady=8, sticky="w")

    # Algorithm selector (only visible during registration)
    algo_label = ttk.Label(frame, text="Algorithm:")
    algo_var = tk.StringVar(value="scrypt")
    algo_menu = ttk.Combobox(frame, textvariable=algo_var, values=["scrypt", "pbkdf2"],
                             width=10, state="readonly")
    algo_menu.current(0)
    algo_label.grid(row=5, column=0, padx=10, pady=5, sticky="e")
    algo_menu.grid(row=5, column=1, padx=10, pady=5, sticky="w")

    # Progress bar (for small fake animation)
    prog = ttk.Progressbar(frame, mode="indeterminate", length=220)
    prog.grid(row=7, column=0, columnspan=3, pady=(10, 5))

    # Action feedback
    result_lbl = tk.Label(frame, text="", bg="#252526", fg="white", font=("Segoe UI", 9))
    result_lbl.grid(row=8, column=0, columnspan=3, pady=(5, 10))

    # Clear fields
    def clear_fields():
        email_entry.delete(0, tk.END)
        pwd_entry.delete(0, tk.END)
        result_lbl.config(text="Fields cleared ✔️", fg="lightgreen")

    # Perform register/login
    def submit():
        email = email_entry.get().strip()
        password = pwd_entry.get()
        if not email or not password:
            messagebox.showerror("Error", "Email and password are required.")
            return

        prog.start(15)
        def task():
            sleep(0.5)
            if mode.get() == "register":
                algo = algo_var.get()
                ok, msg = register_user(email, password, algo)
            else:
                ok, msg = login_user(email, password)
            prog.stop()
            result_lbl.config(text=msg, fg="lightgreen" if ok else "tomato")
            messagebox.showinfo("Result" if ok else "Error", msg)
        threading.Thread(target=task, daemon=True).start()

    # Buttons
    submit_btn = ttk.Button(frame, text="Register User", command=submit)
    submit_btn.grid(row=6, column=0, columnspan=2, pady=(10, 4))
    clear_btn = ttk.Button(frame, text="Clear Fields", command=clear_fields)
    clear_btn.grid(row=6, column=2, pady=(10, 4), padx=(5, 10))

    # Footer
    footer = tk.Label(root, text="Developed for CNS Phase-3 | Secure Password Storage",
                      fg="#999999", bg="#1e1e1e", font=("Segoe UI", 9, "italic"))
    footer.pack(side="bottom", pady=5)

    on_mode_change()
    root.mainloop()

# Run this in a new cell:
# open_gui()


In [14]:
open_gui()

## Validation & testing using different cases

In [47]:
# === Validation Cell: automatic test cases ===

def run_predefined_tests():
    # fresh start
    db.clear_all_users()
    summary = []

    def add_case(name, ok, msg):
        summary.append((name, ok, msg))

    # 1. Normal registration with scrypt
    add_case("Register_scrypt_alice", *register_user("alice@example.com", "StrongPass#1", "scrypt"))

    # 2. Normal registration with PBKDF2
    add_case("Register_pbkdf2_bob", *register_user("bob@example.com", "Another#Pass2", "pbkdf2"))

    # 3. Duplicate registration (same email, different password)
    add_case("Duplicate_email_alice", *register_user("alice@example.com", "NewPass", "scrypt"))

    # 4. Login success for alice
    add_case("Login_success_alice", *login_user("alice@example.com", "StrongPass#1"))

    # 5. Wrong password for alice
    add_case("Login_wrong_password_alice", *login_user("alice@example.com", "WrongPass"))

    # 6. Login unknown user
    add_case("Login_unknown_user", *login_user("charlie@example.com", "Whatever123"))

    # 7. Very long password (edge case)
    long_pwd = "X" * 200
    add_case("Register_long_password", *register_user("long@example.com", long_pwd, "scrypt"))
    add_case("Login_long_password", *login_user("long@example.com", long_pwd))

    # 8. Empty password attempt (should be rejected at app level ideally)
    # here we just show what happens now
    add_case("Register_empty_password", *register_user("empty@example.com", "", "scrypt"))

    # Print summary
    print("=== AUTOMATIC TEST CASE RESULTS ===")
    for name, ok, msg in summary:
        print(f"{name:26} -> ok={ok}, msg={msg}")

    # Show safe view of stored users
    print("\n=== USERS IN DATABASE (safe view) ===")
    for u in list_users():
        print(u)

# Run this to see all the cases & outputs:
run_predefined_tests()


2025-11-03 23:05:43,729 [INFO] Cleared all users from MongoDB collection 'users'.
C:\Users\prash\AppData\Local\Temp\ipykernel_23464\45624763.py:38: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow(),
2025-11-03 23:05:44,087 [INFO] User 'alice@example.com' registered with scrypt.
2025-11-03 23:05:44,381 [INFO] User 'bob@example.com' registered with pbkdf2.
2025-11-03 23:05:44,756 [WARNING] Failed to register 'alice@example.com': Email already exists
2025-11-03 23:05:45,098 [INFO] Login successful for alice@example.com using scrypt.
2025-11-03 23:05:45,452 [WARNING] Login failed for alice@example.com: wrong password.
2025-11-03 23:05:45,454 [WARNING] Login failed for charlie@example.com: User not found.
2025-11-03 23:05:45,825 [INFO] User 'long@example.com' registered with scrypt.
2025-11-03 23:05:46,163 

=== AUTOMATIC TEST CASE RESULTS ===
Register_scrypt_alice      -> ok=True, msg=User 'alice@example.com' registered with scrypt.
Register_pbkdf2_bob        -> ok=True, msg=User 'bob@example.com' registered with pbkdf2.
Duplicate_email_alice      -> ok=False, msg=Failed to register 'alice@example.com': Email already exists
Login_success_alice        -> ok=True, msg=Login successful.
Login_wrong_password_alice -> ok=False, msg=Invalid password.
Login_unknown_user         -> ok=False, msg=User not found.
Register_long_password     -> ok=True, msg=User 'long@example.com' registered with scrypt.
Login_long_password        -> ok=True, msg=Login successful.
Register_empty_password    -> ok=True, msg=User 'empty@example.com' registered with scrypt.

=== USERS IN DATABASE (safe view) ===
{'email': 'alice@example.com', 'algorithm': 'scrypt', 'created_at': datetime.datetime(2025, 11, 3, 17, 35, 44, 86000)}
{'email': 'bob@example.com', 'algorithm': 'pbkdf2', 'created_at': datetime.datetime(2025, 11

In [11]:
# === Final Responsive Modern GUI using customtkinter ===
import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox
import threading
from time import sleep

def open_gui():
    # Global theme and color palette
    ctk.set_appearance_mode("dark")          # "light", "dark", or "system"
    ctk.set_default_color_theme("blue")      # "blue", "green", "dark-blue"

    app = ctk.CTk()
    app.title("Secure Password Storage System")

    # Base window size and minimum size
    app.geometry("950x600")
    app.minsize(850, 550)
    try:
        # Zoom/maximize on Windows; ignore if not supported
        app.state("zoomed")
    except tk.TclError:
        pass

    app.configure(fg_color="#1c1c1c")

    # ====== MAIN CONTAINER with grid-based layout ======
    app.grid_rowconfigure(0, weight=1)
    app.grid_columnconfigure(1, weight=1)   # right side expands

    # -------- LEFT PANEL (Sidebar Info) --------
    sidebar = ctk.CTkFrame(app, corner_radius=0)
    sidebar.grid(row=0, column=0, sticky="nswe")
    sidebar.grid_rowconfigure(5, weight=1)
    sidebar.configure(width=300)

    title = ctk.CTkLabel(
        sidebar,
        text="🔐 Secure Vault",
        font=ctk.CTkFont(family="Segoe UI", size=26, weight="bold"),
    )
    title.grid(row=0, column=0, padx=30, pady=(40, 10), sticky="w")

    subtitle = ctk.CTkLabel(
        sidebar,
        text="Password storage\nwith hashing + salting\n(CNS Phase-3)",
        font=ctk.CTkFont(family="Segoe UI", size=15),
        text_color="#a0a0a0",
        justify="left"
    )
    subtitle.grid(row=1, column=0, padx=30, pady=(0, 20), sticky="w")

    backend = "MongoDB ✅" if not getattr(db, "in_memory", False) else "In-memory (temporary) ⚠️"
    backend_label = ctk.CTkLabel(
        sidebar,
        text=f"Backend: {backend}",
        font=ctk.CTkFont(family="Segoe UI", size=13, slant="italic"),
        text_color="#c0c0c0"
    )
    backend_label.grid(row=2, column=0, padx=30, pady=(0, 10), sticky="w")

    info_text = (
        "• Passwords never stored in plaintext\n"
        "• Uses scrypt / PBKDF2 hashing\n"
        "• Unique random salt per user\n"
        "• Backed by MongoDB / in-memory demo"
    )
    info_label = ctk.CTkLabel(
        sidebar, text=info_text, justify="left",
        font=ctk.CTkFont(family="Consolas", size=12), text_color="#aaaaaa"
    )
    info_label.grid(row=3, column=0, padx=30, pady=(0, 20), sticky="w")

    footer = ctk.CTkLabel(
        sidebar,
        text="Designed by Prashant for CNS Phase-3",
        font=ctk.CTkFont(family="Segoe UI", size=10, slant="italic"),
        text_color="#888888",
    )
    footer.grid(row=6, column=0, padx=30, pady=20, sticky="sw")

    # -------- RIGHT PANEL (Scrollable main form) --------
    right_frame = ctk.CTkScrollableFrame(app)
    right_frame.grid(row=0, column=1, sticky="nsew", padx=20, pady=20)

    # Configure rows to allow expansion if needed
    for i in range(20):
        right_frame.grid_rowconfigure(i, weight=0)
    right_frame.grid_columnconfigure(0, weight=1)

    header = ctk.CTkLabel(
        right_frame,
        text="Account Access Portal",
        font=ctk.CTkFont(family="Segoe UI", size=24, weight="bold"),
    )
    header.grid(row=0, column=0, columnspan=2, pady=(20, 10), sticky="w")

    desc = ctk.CTkLabel(
        right_frame,
        text="Register a new secure account or log in to an existing one.",
        font=ctk.CTkFont(family="Segoe UI", size=14),
        text_color="#b0b0b0"
    )
    desc.grid(row=1, column=0, columnspan=2, pady=(0, 15), sticky="w")

    # Mode selection
    mode_var = ctk.StringVar(value="register")
    mode_switch = ctk.CTkSegmentedButton(
        right_frame,
        values=["Register", "Login"],
        variable=mode_var,
        font=ctk.CTkFont(family="Segoe UI", size=12, weight="bold")
    )
    mode_switch.grid(row=2, column=0, columnspan=2, pady=(0, 20), sticky="w")

    # Email field
    email_label = ctk.CTkLabel(right_frame, text="Email:")
    email_label.grid(row=3, column=0, sticky="w", pady=(5, 0))
    email_entry = ctk.CTkEntry(
        right_frame,
        placeholder_text="example@domain.com",
        width=400
    )
    email_entry.grid(row=4, column=0, columnspan=2, pady=(0, 10), sticky="w")

    # Password field
    pwd_label = ctk.CTkLabel(right_frame, text="Password:")
    pwd_label.grid(row=5, column=0, sticky="w", pady=(5, 0))
    pwd_entry = ctk.CTkEntry(
        right_frame,
        placeholder_text="••••••••",
        show="*",
        width=400
    )
    pwd_entry.grid(row=6, column=0, pady=(0, 10), sticky="w")

    # Show password toggle
    show_pwd_var = ctk.BooleanVar(value=False)
    def toggle_pwd():
        pwd_entry.configure(show="" if show_pwd_var.get() else "*")
    show_chk = ctk.CTkCheckBox(
        right_frame,
        text="Show password",
        variable=show_pwd_var,
        command=toggle_pwd
    )
    show_chk.grid(row=6, column=1, padx=(10, 0), sticky="w")

    # Password strength bar
    strength_label = ctk.CTkLabel(
        right_frame,
        text="Strength: -",
        text_color="#b0b0b0"
    )
    strength_label.grid(row=7, column=0, pady=(5, 0), sticky="w")
    strength_bar = ctk.CTkProgressBar(right_frame, width=400)
    strength_bar.grid(row=8, column=0, columnspan=2, pady=(0, 15), sticky="w")
    strength_bar.set(0)

    def eval_strength(event=None):
        pwd = pwd_entry.get()
        score = 0
        if len(pwd) >= 8: score += 1
        if any(c.isupper() for c in pwd): score += 1
        if any(c.islower() for c in pwd): score += 1
        if any(c.isdigit() for c in pwd): score += 1
        if any(c in "!@#$%^&*()_+-=[]{};:'\",.<>?/\\|" for c in pwd): score += 1
        val = score / 5
        strength_bar.set(val)
        if score <= 2:
            strength_label.configure(text="Strength: Weak", text_color="#ff6060")
        elif score == 3:
            strength_label.configure(text="Strength: Medium", text_color="#ffd966")
        else:
            strength_label.configure(text="Strength: Strong", text_color="#66ff99")

    pwd_entry.bind("<KeyRelease>", eval_strength)

    # Algorithm dropdown (for register mode)
    algo_label = ctk.CTkLabel(right_frame, text="Hashing Algorithm:")
    algo_label.grid(row=9, column=0, pady=(5, 0), sticky="w")
    algo_var = ctk.StringVar(value="scrypt")
    algo_dropdown = ctk.CTkOptionMenu(
        right_frame,
        values=["scrypt", "pbkdf2"],
        variable=algo_var,
        width=180
    )
    algo_dropdown.grid(row=10, column=0, pady=(0, 15), sticky="w")

    # Progress and status
    prog = ctk.CTkProgressBar(right_frame, mode="indeterminate", width=400)
    prog.grid(row=12, column=0, columnspan=2, pady=(5, 5), sticky="w")

    status_lbl = ctk.CTkLabel(
        right_frame,
        text="Ready.",
        text_color="#aaaaaa",
        font=ctk.CTkFont(size=12, slant="italic")
    )
    status_lbl.grid(row=13, column=0, columnspan=2, pady=(0, 10), sticky="w")

    # Handle mode changes (hide algo in login mode)
    def update_mode(*args):
        if mode_var.get().lower() == "login":
            algo_label.grid_remove()
            algo_dropdown.grid_remove()
        else:
            algo_label.grid(row=9, column=0, pady=(5, 0), sticky="w")
            algo_dropdown.grid(row=10, column=0, pady=(0, 15), sticky="w")
    mode_var.trace_add("write", update_mode)
    update_mode()

    # Clear fields
    def clear_fields():
        email_entry.delete(0, "end")
        pwd_entry.delete(0, "end")
        strength_bar.set(0)
        strength_label.configure(text="Strength: -", text_color="#b0b0b0")
        status_lbl.configure(text="Fields cleared.", text_color="#b0b0b0")

    # Submit (register or login)
    def do_action():
        email = email_entry.get().strip()
        password = pwd_entry.get()
        if not email or not password:
            status_lbl.configure(text="Email & password required.", text_color="#ff6060")
            messagebox.showerror("Error", "Email and password are required.")
            return

        def worker():
            prog.start()
            status_lbl.configure(text="Processing...", text_color="#cccccc")
            sleep(0.3)

            if mode_var.get().lower() == "register":
                algo = algo_var.get()
                ok, msg = register_user(email, password, algo)
            else:
                ok, msg = login_user(email, password)

            prog.stop()
            status_lbl.configure(text=msg, text_color="#66ff99" if ok else "#ff6060")
            if ok:
                messagebox.showinfo("Success", msg)
            else:
                messagebox.showerror("Error", msg)

        threading.Thread(target=worker, daemon=True).start()

    # Buttons row
    btn_frame = ctk.CTkFrame(right_frame, fg_color="transparent")
    btn_frame.grid(row=11, column=0, columnspan=2, pady=(0, 10), sticky="w")

    submit_btn = ctk.CTkButton(
        btn_frame,
        text="Submit",
        width=140,
        command=do_action
    )
    submit_btn.pack(side="left", padx=(0, 10))

    clear_btn = ctk.CTkButton(
        btn_frame,
        text="Clear",
        width=100,
        fg_color="#444444",
        hover_color="#555555",
        command=clear_fields
    )
    clear_btn.pack(side="left")

    app.mainloop()

# To open the GUI, run in a new cell:
# open_gui()


In [13]:
open_gui()

C:\Users\prash\AppData\Local\Temp\ipykernel_14632\45624763.py:38: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow(),
2025-11-04 03:30:10,458 [INFO] User 'aryan.chaurasia@gmail.com' registered with pbkdf2.
2025-11-04 03:30:16,742 [INFO] Login successful for aryan.chaurasia@gmail.com using pbkdf2.
